In [1]:
import time
import random
import requests
import pandas as pd
import time
import random
import yaml
import json
import torch
import datetime as dt
import re

with open("shopee.yaml") as f:
    cfg=yaml.load(f,Loader=yaml.SafeLoader)
    cfg['device']=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"### Loading config {cfg}")

### Loading config {'P001': './json/iphone15promax.json', 'P002': './json/samsungs23ultra.json', 'P003': './json/samsungs24ultra.json', 'device': device(type='cpu')}


In [86]:
def get_info(data):
  data_list = [{'ItemID': item['item_basic']['itemid'],
                'ShopID': item['item_basic']['shopid'],
                'Name': item['item_basic']['name'],
                # 'stock':item['item_basic']['stock'],
                # 'ctime':item['item_basic']['ctime'],
                # 'sold':item['item_basic']['sold'],
                'Quantity':item['item_basic']['historical_sold'],
                # 'liked_count':item['item_basic']['liked_count'],
                'ReviewCount':item['item_basic']['item_rating']['rating_count'][0],
                'Price':item['item_basic']['price'],
                # 'options':item['item_basic']['tier_variations'][0]['options'],
                'StarRating':item['item_basic']['item_rating']['rating_star']} 
              for item in data['items']]
  df=pd.DataFrame(data_list,index=None)
  return df


# Filter Products for Samsung
def filter_products(data,filter1,filter2):
    
    # Remove Product's Name that not related (s24, fe, iphone 12)
    names_lower = data['Name'].str.lower()
    pattern = re.compile('|'.join(filter1))
    indices = ~names_lower.str.contains(pattern)
    data=data[indices]
  
    # Remove Product's Name that not contain related words (ex: iphone 15 pro max/ samsung s23 ultra)
    names_lower = data['Name'].str.lower()
    pattern = re.compile(r'(?=.*{})'.format(')(?=.*'.join(filter2)), flags=re.IGNORECASE)
    indices = ~names_lower.str.contains(pattern)
    data=data[indices]

    data = data[data['Quantity']!=0]
    data.reset_index(inplace=True)
    return data

In [93]:
products=['P001','P002','P003']
save_name = ['iphone_15_pro_max_info','galaxy_s23_ultra_info']
# ['ProductID', 'PlatformID', 'ShopID', 'Price', 'Quantity', 'Reviews count', 'Star rating', 'Crawl date']
df = pd.DataFrame()
filter_1 = [['máy tính','plus','samsung','galaxy','ốp lưng','ốp','iphone 11','iphone 12','iphone 13','iphone 14'],['a54','s24','s22','z','fold','flip4','s21','note','flip5','fe','s24+','iphone','ốp','ốp lưng']]
filter_2 = [['iphone'],['s24','ultra']]

for idx,file in enumerate(products[:-2]):
    with open(cfg[file],'r',encoding='utf-8') as f:
        data=json.load(f)
        print(f'Saving data of {file}')
        df_temp = get_info(data)
        df_temp['PlatformID'] = 'PL03'
        df_temp['ProductID'] = file
        df_temp['CrawlDate'] = dt.datetime.today()
        df = filter_products(df_temp,filter_1[idx],filter_2[idx])
        df.to_csv(f'../../Data/Shopee/{save_name[idx]}.csv',index=False)


Saving data of P001


In [ ]:
# iphone = pd.read_csv('..\..\Data\Shopee\iphone_15_pro_max_info.csv')
# lst_filter = ['promax','pro max']
# filtered = iphone['Name'].str.contains('|'.join(lst_filter),case=False)
# iphone=iphone[filtered]
# iphone.drop(columns=['index'],inplace=True)
# iphone.to_csv('..\..\Data\Shopee\iphone_15_pro_max_info.csv',index=False)

In [100]:
# MAIN
def get_reviews(shop_id,item_id,prodid):
    url = "https://shopee.com.my/api/v2/item/get_ratings"
    limit = 1
    offset = 0
    reviews = []
    
    session = requests.Session()
    session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})
    
    while True:
        params = {
            "itemid": item_id,
            "shopid": shop_id,
            "offset": offset,
            "limit": limit,
            "filter": "0",
            "flag": "0",
            "sort": "0",
            "append": "0",
            "before_bundle": "",
            "language": "en",
        }
    
        response = session.get(url, params=params).json()
        if response["error"]:
            print(f"Error: {response['error']}")
            break
        elif response["data"]["ratings"]:
            for rating in response["data"]["ratings"][:201]:
                model_name = rating.get('product_items')[0].get('model_name')
                rating['model_name'] = model_name
                reviews.append(rating)
                offset += limit
        else:
            break
        
    df = pd.DataFrame(reviews) 

    print(f"Retrieved {len(reviews)} reviews.")
    df = df[[ 'itemid','ctime','shopid', 'comment', 'rating_star']]
    df.rename(columns={"itemid": "ItemID",'shopid':'ShopID','ctime':'DateReview', 'comment':'Review','rating_star':'StarRating'},inplace=True)
    df['ProductID'] = prodid
    df['PlatformID'] = 'PL03'
    df['DateReview'] = pd.to_datetime(df['DateReview'],unit='s').dt.date

    # print(f'Crawl shop: {shop_id}, item{item_id} completed')
    # df.to_excel(f'Shopee_shopid{shop_id}.xlsx', index=False)
    return df


In [101]:
import os
import pandas as pd
import requests
# data_rw = pd.DataFrame()
prod_name = ['galaxy_s23_ultra','iphone_15_pro_max']

for name in prod_name:
    path = f'../../Data/Shopee/{name}_info.csv'
    if os.path.isfile(path):
        shopee_prod_info = pd.read_csv(path)
    else:
        continue
    
    shopee_prod_info = shopee_prod_info[shopee_prod_info['ReviewCount']!=0]
    data_rw = pd.DataFrame()

    for itemid, shopid,prodid in zip(shopee_prod_info['ItemID'],shopee_prod_info['ShopID'],shopee_prod_info['ProductID']):
        df_rw = get_reviews(shopid,itemid,prodid)
        data_rw = pd.concat([data_rw,df_rw],ignore_index=True)
    data_rw.to_csv(f'../../Data/Shopee/{name}_review.csv',index=False)
    print(f'Crawled {name} reviews')

Retrieved 48 reviews.
Retrieved 32 reviews.
Retrieved 13 reviews.
Retrieved 46 reviews.
Retrieved 4 reviews.
Retrieved 5 reviews.
Retrieved 97 reviews.
Retrieved 89 reviews.
Retrieved 6 reviews.
Retrieved 1 reviews.
Retrieved 2 reviews.
Retrieved 53 reviews.
Retrieved 8 reviews.
Retrieved 89 reviews.
Retrieved 25 reviews.
Retrieved 9 reviews.
Retrieved 1 reviews.
Retrieved 2 reviews.
Retrieved 1 reviews.
Retrieved 1 reviews.
Retrieved 33 reviews.
Retrieved 22 reviews.
Retrieved 21 reviews.
Crawled galaxy_s23_ultra reviews


# Done

In [ ]:
# import os
# import csv
# def concat_csv_review(directory):
#     csv_files = [file for file in os.listdir(directory) if file.startswith("Shopee_shopid")]
#     if not csv_files:
#         return "no CSV files found."
    
#     dfs=[]

#     for file_name in csv_files:
#         file_path=os.path.join(directory,file_name)
#         dfs.append(pd.read_csv(file_path))

#     df_sum = pd.concat(dfs,ignore_index=True)

#     df_sum.drop(['orderid','cmtid','ctime','userid',,'like_count','model_name'],axis=1,inplace=True)
#     df_sum.to_csv()
#     return df_sum

In [24]:
len(df)

18

In [ ]:
import requests
import pandas as pd

url = "https://shopee.com.my/api/v2/item/get_ratings"
shop_id = "308461157"
item_id = "23354704062"
limit = 1
offset = 0
reviews = []

session = requests.Session()
session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})



while True:
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "offset": offset,
        "limit": limit,
        "filter": "0",
        "flag": "0",
        "sort": "0",
        "append": "0",
        "before_bundle": "",
        "language": "en",
    }

    response = session.get(url, params=params).json()
    if response["error"]:
        print(f"Error: {response['error']}")
        break
    elif response["data"]["ratings"]:
        print(offset)
        print (response["data"]["ratings"])

        for rating in response["data"]["ratings"]:
            model_name = rating.get('product_items')[0].get('model_name')
            rating['model_name'] = model_name
            reviews.append(rating)
            offset += limit

            time.sleep(random.randint(2,5))  
    else:
        break

df = pd.DataFrame(reviews)

print(f"Retrieved {len(reviews)} reviews.")
# extract 'skin suitability' and 'absorption' from 'comment' column
df[['skin_suitability', 'absorption']] = df['comment'].str.extract('Skin Suitability:(.*?)\\nAbsorption:(.*?)\\n', expand=True)

df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'rating', 'userid', 'shopid', 'comment', 'rating_star', 'skin_suitability', 'absorption', 'model_name']]

print('success')

df.to_csv('ShopeeScrap1.csv', index=False)


In [ ]:
import requests
import pandas as pd
import time
import random

def get_reviews(shop_id,item_id):
    url = "https://shopee.com.my/api/v2/item/get_ratings"
    limit = 1
    offset = 0
    reviews = []
    
    session = requests.Session()
    session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "offset": offset,
        "limit": limit,
        "filter": "0",
        "flag": "0",
        "sort": "0",
        "append": "0",
        "before_bundle": "",
        "language": "en",
    }
    response = session.get(url, params=params).json()
    print(response)
    for rating in response["data"]["ratings"][:10]:
        model_name = rating.get('product_items')[0].get('model_name')
        rating['model_name'] = model_name
        reviews.append(rating)
        offset += limit

    df = pd.DataFrame(reviews)
    print(f"Retrieved {len(reviews)} reviews.")
    
    df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'userid', 'shopid', 'comment', 'rating_star','like_count',  'model_name']]
    df.to_excel('temp.xlsx', index=False)

    
shop_id = "288286284"
item_id = "21993222972"
get_reviews(shop_id,item_id)

{'error_msg': None, 'data': {'ratings': [{'orderid': 153680409319298, 'itemid': 21993222972, 'cmtid': 13284042161, 'ctime': 1700535964, 'rating': 1, 'userid': 330045429, 'shopid': 288286284, 'comment': 'Chất lượng sản phẩm:tốt\nĐúng với mô tả:tốt\nTính năng nổi bật:màu titan tự nhiên\n\nShopdunk vận chuyển nhanh chóng, rất tốt. Mua ở đây yên tâm hơn mua ở apple flagship store nhiều. Sẽ ủng hộ mua của shopdunk', 'rating_star': 5, 'status': 2, 'mtime': 1704926407, 'editable': 0, 'opt': 2, 'filter': 7, 'mentioned': [], 'is_hidden': False, 'can_follow_up': None, 'follow_up': None, 'submit_time': 1700535964, 'author_username': 'h*****t', 'author_portrait': '', 'author_shopid': 330025854, 'anonymous': True, 'images': ['vn-11134103-7r98o-loc17m6vu9dj4c', 'vn-11134103-7r98o-loc17m6vx2if65'], 'videos': [], 'product_items': [{'itemid': 21993222972, 'shopid': 288286284, 'name': 'Apple iPhone 15 Pro Max 256GB', 'image': 'vn-11134207-7r98o-llt6eul6d7tr35', 'is_snapshot': 1, 'snapshotid': 1551599355

In [33]:
# product/
def get_reviews(shop_id,item_id):
    url = "https://shopee.com.my/api/v2/item/get_item_detail"
    limit = 1
    offset = 0
    reviews = []
    
    session = requests.Session()
    session.headers.update({"Cookie": "_gcl_au=xxxx; SPC_IA=-1; SPC_EC=-; SPC_F=xxxx; SPC_U=-; SPC_T_ID=xxxx; SPC_T_IV=xxxx; SPC_SI=xxxx; _ga=xxxx; _gid=xxxx; cto_lwid=xxxx; _fbp=xxxx; _hjid=xxxx; SPC_SIxxxx=xxxx"})
    
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "offset": offset,
        "limit": limit,
        "filter": "0",
        "flag": "0",
        "sort": "0",
        "append": "0",
        "before_bundle": "",
        "language": "en",
    }
    keyword_search = 'iphone 15 pro max'
    url = "https://shopee.sg/api/v2/search_items/?by=relevancy&keyword={}&limit=100&newest=0&order=desc&page_type=search".format(keyword_search)
    response = session.get(url, params=params).json()
    print(response)
    
# can change "relevancy" to "latest": to sort by latest products instead

# Shopee API request
    #     if response["error"]:
    #         print(f"Error: {response['error']}")
    #         break
    #     elif response["data"]["ratings"]:
    #         print(offset)
    #         print (response["data"]["ratings"])
    
    #         for rating in response["data"]["ratings"]:
    #             model_name = rating.get('product_items')[0].get('model_name')
    #             rating['model_name'] = model_name
    #             reviews.append(rating)
    #             offset += limit
    
    #             time.sleep(random.randint(2,5))  
    #     else:
    #         break
    
    # df = pd.DataFrame(reviews)
    
    # print(f"Retrieved {len(reviews)} reviews.")
    # # extract 'skin suitability' and 'absorption' from 'comment' column
    # df[['skin_suitability', 'absorption']] = df['comment'].str.extract('Skin Suitability:(.*?)\\nAbsorption:(.*?)\\n', expand=True)
    
    # # select columns to keep in the final CSV file
    # df = df[['orderid', 'itemid', 'cmtid', 'ctime', 'rating', 'userid', 'shopid', 'comment', 'rating_star', 'skin_suitability', 'absorption', 'model_name']]
    # # df = df.to_string(index=False)
    
    # print('success')
    
    # # save the DataFrame as a CSV file
    # df.to_csv('ShopeeScrap1.csv', index=False)
shop_id = "288286284"
item_id = "21993222972"
get_reviews(shop_id,item_id)

{'error': 'error_not_found'}
